In [1]:
!pip install ultralytics

  Using cached numpy-1.26.4-cp311-cp311-win_amd64.whl.metadata (61 kB)
  Using cached matplotlib-3.9.2-cp311-cp311-win_amd64.whl.metadata (11 kB)
  Using cached opencv_python-4.10.0.84-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached pillow-10.4.0-cp311-cp311-win_amd64.whl.metadata (9.3 kB)
  Using cached scipy-1.14.1-cp311-cp311-win_amd64.whl.metadata (60 kB)
  Using cached torch-2.4.1-cp311-cp311-win_amd64.whl.metadata (27 kB)
  Using cached torchvision-0.19.1-cp311-cp311-win_amd64.whl.metadata (6.1 kB)
  Using cached tqdm-4.66.5-py3-none-any.whl.metadata (57 kB)
  Using cached py_cpuinfo-9.0.0-py3-none-any.whl.metadata (794 bytes)
  Using cached pandas-2.2.2-cp311-cp311-win_amd64.whl.metadata (19 kB)
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached ultralytics_thop-2.0.6-py3-none-any.whl.metadata (9.1 kB)
  Using cached contourpy-1.3.0-cp311-cp311-win_amd64.whl.metadata (5.4 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  U

In [ ]:
import cv2

video_path = r'C:\Users\tahay\1-GeneralPythonFolder\road_intersection.mp4'
cap = cv2.VideoCapture(video_path)

In [2]:
import cv2

# Video dosyasının yolu
video_path = r'C:\Users\tahay\1-GeneralPythonFolder\road_intersection.mp4'
cap = cv2.VideoCapture(video_path)

# Noktaların konulacağı aralık
interval = 50  # Piksel cinsinden aralık
dot_radius = 10  # Noktaların yarıçapı
dot_color = (0, 0, 255)  # BGR formatında kırmızı renk
text_color = (255, 255, 255)  # BGR formatında beyaz renk
font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 0.3  # Font boyutunu küçülttük
thickness = 1

# Noktaların koordinatlarını tutacak liste
dot_coordinates = []

frame_index = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    height, width, _ = frame.shape

    # Noktaları ve sayıları çizin
    count = 0
    for y in range(interval, height, interval):
        for x in range(interval, width, interval):
            dot_coordinates.append((x, y))
            cv2.circle(frame, (x, y), dot_radius, dot_color, -1)
            cv2.putText(frame, str(count), (x - 10, y - 15), font, font_scale, text_color, thickness)
            cv2.putText(frame, f'({x},{y})', (x - 10, y + 15), font, font_scale, text_color, thickness)
            count += 1

    cv2.imshow('Frame with Dots', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    frame_index += 1

cap.release()
cv2.destroyAllWindows()




KeyboardInterrupt



In [ ]:
import cv2
import numpy as np

# Video dosyasının yolu
video_path = r'C:\Users\tahay\1-GeneralPythonFolder\road_intersection.mp4'
cap = cv2.VideoCapture(video_path)

# Bölge koordinatları (dört köşe noktaları)
regions = [
    [(250, 350), (150, 150),(300, 150), (550, 340)],
    [(750, 550), (1100, 500), (1250, 600),(1250, 710),(1150, 710), (950, 710)],
    [(1050, 400), (950, 350), (1250, 310), (1250, 380)],
    [(250, 450), (300, 550), (10, 600), (10, 460)]
]

# Nokta özellikleri
dot_radius = 10  # Noktaların yarıçapı
dot_color = (0, 0, 255)  # BGR formatında kırmızı renk
text_color = (255, 255, 255)  # BGR formatında beyaz renk
font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 0.5  # Font boyutunu küçülttük
thickness = 1

# Fonksiyon: Dört nokta ile bir dikdörtgen çizer
def draw_polygon(frame, points, color, thickness):
    points = np.array(points, np.int32)
    points = points.reshape((-1, 1, 2))
    cv2.polylines(frame, [points], isClosed=True, color=color, thickness=thickness)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Bölgeleri çiz
    for region in regions:
        draw_polygon(frame, region, (0, 255, 0), 2)
        for idx, point in enumerate(region):
            cv2.circle(frame, point, dot_radius, dot_color, -1)
            cv2.putText(frame, f'({point[0]},{point[1]})', (point[0] - 20, point[1] - 10), font, font_scale, text_color, thickness)

    cv2.imshow('Frame with Regions and Dots', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [3]:
import cv2
import numpy as np
from ultralytics import YOLO

# YOLOv8 modelini yükle
model = YOLO('yolov8n.pt')

# Video dosyasının yolu
video_path = r'C:\Users\tahay\1-GeneralPythonFolder\road_intersection.mp4'
cap = cv2.VideoCapture(video_path)

# Video yazıcıyı başlat
output_path = r'C:\Users\tahay\1-GeneralPythonFolder\output.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

# Bölge isimleri ve koordinatları (dört köşe noktaları)
regions = [
    {"name": "Region1", "points": [(250, 350), (150, 150), (300, 150), (550, 340)]},
    {"name": "Region2", "points": [(750, 550), (1100, 500), (1250, 600), (1250, 710), (1150, 710), (950, 710)]},
    {"name": "Region3", "points": [(1050, 400), (950, 350), (1250, 310), (1250, 380)]},
    {"name": "Region4", "points": [(250, 450), (300, 550), (10, 600), (10, 460)]}
]

# Nokta özellikleri
dot_radius = 10  # Noktaların yarıçapı
dot_color = (0, 0, 255)  # BGR formatında kırmızı renk
text_color = (255, 255, 255)  # BGR formatında beyaz renk
text_bg_color = (0, 0, 0)  # BGR formatında siyah arka plan
font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 0.7  # Font boyutunu büyüttük
font_thickness = 2  # Yazı kalınlığını artırdık
thickness = 1

# Fonksiyon: Dört nokta ile bir dikdörtgen çizer
def draw_polygon(frame, points, color, thickness):
    points = np.array(points, np.int32)
    points = points.reshape((-1, 1, 2))
    cv2.polylines(frame, [points], isClosed=True, color=color, thickness=thickness)

# Fonksiyon: Bir noktanın bir bölge içinde olup olmadığını kontrol eder
def point_in_polygon(point, polygon):
    x, y = point
    return cv2.pointPolygonTest(np.array(polygon, dtype=np.int32), (x, y), False) >= 0

# Fonksiyon: Bir bölgenin merkezini hesaplar
def get_region_center(region):
    points = np.array(region)
    center_x = int(np.mean(points[:, 0]))
    center_y = int(np.mean(points[:, 1]))
    return (center_x, center_y)

# Araç sınıfı id'si
vehicle_classes = [2, 3, 5, 7]  # COCO dataset sınıf id'leri: Car, Motorcycle, Bus, Truck

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # YOLOv8 ile araç tespiti
    results = model(frame)
    
    # Algılanan araçların koordinatlarını al
    detected_vehicles = []
    for result in results:
        for box in result.boxes:
            cls_id = int(box.cls[0])
            if cls_id in vehicle_classes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                center = ((x1 + x2) // 2, (y1 + y2) // 2)
                detected_vehicles.append(center)
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
                cv2.putText(frame, f'{model.names[cls_id]}', (x1, y1 - 10), font, font_scale, text_color, font_thickness)

    # Bölgeleri çiz ve araçları say
    max_vehicle_count = 0
    max_region_name = ""
    for region in regions:
        draw_polygon(frame, region["points"], (0, 255, 0), 2)
        vehicle_count = 0
        for vehicle in detected_vehicles:
            if point_in_polygon(vehicle, region["points"]):
                vehicle_count += 1
        
        region_center = get_region_center(region["points"])
        for idx, point in enumerate(region["points"]):
            cv2.circle(frame, point, dot_radius, dot_color, -1)
            cv2.putText(frame, f'({point[0]},{point[1]})', (point[0] - 20, point[1] - 10), font, font_scale, text_color, thickness)
        
        # Bölge ismi ve araç sayısını yazdır
        text = f'{region["name"]}- Vehicles: {vehicle_count}'
        text_size, _ = cv2.getTextSize(text, font, font_scale, font_thickness)
        text_w, text_h = text_size
        cv2.rectangle(frame, (region_center[0] - text_w // 2 - 5, region_center[1] - text_h // 2 - 5), 
                      (region_center[0] + text_w // 2 + 5, region_center[1] + text_h // 2 + 5), text_bg_color, -1)
        cv2.putText(frame, text, (region_center[0] - text_w // 2, region_center[1] + text_h // 2), font, font_scale, (0, 255, 0), font_thickness)

        # En fazla araç bulunan bölgeyi belirle
        if vehicle_count > max_vehicle_count:
            max_vehicle_count = vehicle_count
            max_region_name = region["name"]

    # Videonun ortasında en fazla araç bulunan bölgenin adını yazdır
    if max_region_name:
        frame_center = (frame.shape[1] // 2, frame.shape[0] // 2)
        priority_text = f'{max_region_name} - Priority'
        text_size, _ = cv2.getTextSize(priority_text, font, font_scale, font_thickness)
        text_w, text_h = text_size
        cv2.rectangle(frame, (frame_center[0] - text_w // 2 - 5, frame_center[1] - text_h // 2 - 5), 
                      (frame_center[0] + text_w // 2 + 5, frame_center[1] + text_h // 2 + 5), text_bg_color, -1)
        cv2.putText(frame, priority_text, (frame_center[0] - text_w // 2, frame_center[1] + text_h // 2), font, font_scale, (0, 255, 0), font_thickness)

    # Çıktı videosuna kare ekle
    out.write(frame)

    cv2.imshow('Frame with Regions and Dots', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()



0: 384x640 12 cars, 1 fire hydrant, 55.5ms
Speed: 4.0ms preprocess, 55.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 cars, 1 fire hydrant, 59.1ms
Speed: 3.0ms preprocess, 59.1ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 13 cars, 1 fire hydrant, 55.5ms
Speed: 1.0ms preprocess, 55.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 13 cars, 1 fire hydrant, 53.5ms
Speed: 2.0ms preprocess, 53.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 13 cars, 1 fire hydrant, 51.5ms
Speed: 2.0ms preprocess, 51.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 cars, 1 fire hydrant, 53.1ms
Speed: 2.0ms preprocess, 53.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 cars, 1 fire hydrant, 59.0ms
Speed: 2.0ms preprocess, 59.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1